## Geocoding India's cities
### Step 1: This notebook will read in the cities.csv and geocode all the city names from the "City" column.
### Step 2: The coordinates will be merged with the original cities.csv file and be read back into the india-aqi-scraper.ipynb file

In [14]:
#imports
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os

In [15]:
load_dotenv() 

API_KEY = os.getenv("GOOGLE_GEOCODING_API")

In [16]:

def geocode_city(city_name):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": city_name, "key": API_KEY}
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        print(f"Skipped {city_name}: {data.get('error_message', data['status'])}")
        return None, None

In [17]:
df = pd.read_csv("cities.csv")
unique_cities = df['City'].unique()
print(f"Found {len(unique_cities)} unique cities to geocode.")

Found 242 unique cities to geocode.


In [18]:
results = []
for city in unique_cities:
    lat, lng = geocode_city(city)
    results.append({"City": city, "Lat": lat, "Lng": lng})
    time.sleep(0.1)  # Rate limiting

In [19]:
results

[{'City': 'Agra', 'Lat': 27.1766701, 'Lng': 78.00807449999999},
 {'City': 'Ahmedabad', 'Lat': 23.022505, 'Lng': 72.5713621},
 {'City': 'Ahmednagar', 'Lat': 19.0948287, 'Lng': 74.74797889999999},
 {'City': 'Aizawl', 'Lat': 23.7307175, 'Lng': 92.71731059999999},
 {'City': 'Ajmer', 'Lat': 26.4498954, 'Lng': 74.6399163},
 {'City': 'Akola', 'Lat': 20.7002159, 'Lng': 77.0081678},
 {'City': 'Alwar', 'Lat': 27.5529907, 'Lng': 76.6345735},
 {'City': 'Amaravati', 'Lat': 16.5130887, 'Lng': 80.5164523},
 {'City': 'Ambala', 'Lat': 30.3752011, 'Lng': 76.782122},
 {'City': 'Amravati', 'Lat': 20.9319821, 'Lng': 77.7523039},
 {'City': 'Amritsar', 'Lat': 31.6339793, 'Lng': 74.8722642},
 {'City': 'Anantapur', 'Lat': 14.6824455, 'Lng': 77.6017089},
 {'City': 'Angul', 'Lat': 20.8379124, 'Lng': 85.09779139999999},
 {'City': 'Ankleshwar', 'Lat': 21.6264236, 'Lng': 73.0151984},
 {'City': 'Araria', 'Lat': 26.1324689, 'Lng': 87.4528067},
 {'City': 'Arrah', 'Lat': 25.5541358, 'Lng': 84.6664797},
 {'City': 'Asans

In [21]:
coords_df = pd.DataFrame(results)
# Joining this with the original dataframe with city names
final_df = pd.merge(df, coords_df, on="City", how="left")

# Saving as a df to be used in the india-aqi-scraper.ipynb file
final_df.to_csv("city_coords.csv", index=False)